In [ ]:
!pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.2 MB/s eta 0:00:00


In [ ]:
!pip install sktime[all_extras]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.8/875.8 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 KB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━

In [1]:
from sktime.classification.hybrid import HIVECOTEV1

In [2]:
from sktime.datasets import load_from_tsfile_to_dataframe as load_ts

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import KFold

In [5]:
X5_small, y5_small = load_ts("ECGFiveDays_TRAIN.ts")

FileNotFoundError: [Errno 2] No such file or directory: 'ECGFiveDays_TRAIN.ts'

In [5]:
X5_test, y5_test = load_ts("ECGFiveDays/ECGFiveDays_TEST.ts")
X5_small, y5_small = load_ts("ECGFiveDays/ECGFiveDays_TRAIN.ts")
X200_train, y200_train = load_ts("ECG200/ECG200_TRAIN.ts")
X5000_train, y5000_train = load_ts("ECG5000/ECG5000_TRAIN.ts")
X200_test, y200_test = load_ts("ECG200/ECG200_TEST.ts")
X5000_test, y5000_test = load_ts("ECG5000/ECG5000_TEST.ts")

In [22]:
df = pd.read_csv('mitbih_train.csv')
X_ECG = np.array(df.iloc[80000:87553, :-1])
print(len(X_ECG))
y_ECG = np.array(df.iloc[80000:87553 ,-1])
print(len(y_ECG))

7553
7553


In [13]:
df.iloc[:,-1].value_counts()

0.0    72470
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: 0.000000000000000000e+00.88, dtype: int64

In [21]:
df = pd.concat([df.iloc[500:6500],
           df.iloc[73066:73276],
           df.iloc[75222:75755],
           df.iloc[80490:80561],
           df.iloc[81150:81775]])
X_ECG = np.array(df.iloc[:, :-1])
print(len(X_ECG))
y_ECG = np.array(df.iloc[ :,-1])
print(len(y_ECG))

NameError: name 'df' is not defined

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
def map_to_scores(x):
    if x == 1:
        return torch.tensor([1., 0.])
    if x == 2:
        return torch.tensor([0., 1.])

class TSCDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return torch.tensor([self.x.iloc[i][0]]).float(), map_to_scores(float(self.y[i]))

In [7]:
from datetime import datetime

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.utils import check_random_state

from sktime.classification.base import BaseClassifier
from sktime.classification.dictionary_based import ContractableBOSS
from sktime.classification.interval_based import (
    RandomIntervalSpectralEnsemble,
    TimeSeriesForestClassifier,
)
from sktime.classification.shapelet_based import ShapeletTransformClassifier

class TSCDataset_MY(Dataset):
    def __init__(self, X):
        self.x = X
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return torch.tensor([self.x.iloc[i][0]]).float()

In [8]:
class HIVECOTEV1_MY(HIVECOTEV1):
    _tags = {
        "capability:multithreading": True,
        "classifier_type": "hybrid",
    }

    def __init__(
        self,
        stc_params=None,
        tsf_params=None,
        rise_params=None,
        cboss_params=None,
        verbose=0,
        n_jobs=1,
        random_state=None,
    ):
        self.stc_params = stc_params
        self.tsf_params = tsf_params
        self.rise_params = rise_params
        self.cboss_params = cboss_params

        self.verbose = verbose
        self.n_jobs = n_jobs
        self.random_state = random_state

        self.stc_weight_ = 0
        self.tsf_weight_ = 0
        self.rise_weight_ = 0
        self.cboss_weight_ = 0

        self.cnn_weight_ = 0

        self._stc_params = stc_params
        self._tsf_params = tsf_params
        self._rise_params = rise_params
        self._cboss_params = cboss_params
        self._stc = None
        self._tsf = None
        self._rise = None
        self._cboss = None

        self.cnn = None

        super(HIVECOTEV1, self).__init__()

    def _fit(self, X, y):
        """Fit HIVE-COTE 1.0 to training data.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The training data.
        y : array-like, shape = [n_instances]
            The class labels.
        Returns
        -------
        self :
            Reference to self.
        Notes
        -----
        Changes state by creating a fitted model that updates attributes
        ending in "_" and sets is_fitted flag to True.
        """
        # Default values from HC1 paper
        if self.stc_params is None:
            self._stc_params = {"transform_limit_in_minutes": 120}
        if self.tsf_params is None:
            self._tsf_params = {"n_estimators": 500}
        if self.rise_params is None:
            self._rise_params = {"n_estimators": 500}
        if self.cboss_params is None:
            self._cboss_params = {}

        # Cross-validation size for TSF and RISE
        cv_size = 10
        _, counts = np.unique(y, return_counts=True)
        min_class = np.min(counts)
        if min_class < cv_size:
            cv_size = min_class

        # Build STC
        self._stc = ShapeletTransformClassifier(
            **self._stc_params,
            save_transformed_data=True,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._stc.fit(X, y)

        if self.verbose > 0:
            print("STC ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find STC weight using train set estimate
        train_probs = self._stc._get_train_probs(X, y)
        train_preds = self._stc.classes_[np.argmax(train_probs, axis=1)]
        self.stc_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "STC train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("STC weight = " + str(self.stc_weight_))  # noqa

        # Build TSF
        self._tsf = TimeSeriesForestClassifier(
            **self._tsf_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._tsf.fit(X, y)

        if self.verbose > 0:
            print("TSF ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find TSF weight using train set estimate found through CV
        train_preds = cross_val_predict(
            TimeSeriesForestClassifier(
                **self._tsf_params, random_state=self.random_state
            ),
            X=X,
            y=y,
            cv=cv_size,
            n_jobs=self._threads_to_use,
        )
        self.tsf_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "TSF train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("TSF weight = " + str(self.tsf_weight_))  # noqa

        # Build RISE
        self._rise = RandomIntervalSpectralEnsemble(
            **self._rise_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._rise.fit(X, y)

        if self.verbose > 0:
            print("RISE ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find RISE weight using train set estimate found through CV
        train_preds = cross_val_predict(
            RandomIntervalSpectralEnsemble(
                **self._rise_params,
                random_state=self.random_state,
            ),
            X=X,
            y=y,
            cv=cv_size,
            n_jobs=self._threads_to_use,
        )
        self.rise_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "RISE train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("RISE weight = " + str(self.rise_weight_))  # noqa

        # Build cBOSS
        self._cboss = ContractableBOSS(
            **self._cboss_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._cboss.fit(X, y)

        # Find cBOSS weight using train set estimate
        train_probs = self._cboss._get_train_probs(X, y)
        train_preds = self._cboss.classes_[np.argmax(train_probs, axis=1)]
        self.cboss_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "cBOSS (estimate included)",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("cBOSS weight = " + str(self.cboss_weight_))  # noqa

        return self

    def _predict(self, X) -> np.ndarray:
        """Predicts labels for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predictions for.
        Returns
        -------
        y : array-like, shape = [n_instances]
            Predicted class labels.
        """
        rng = check_random_state(self.random_state)
        return np.array(
            [
                self.classes_[int(rng.choice(np.flatnonzero(prob == prob.max())))]
                for prob in self.predict_proba(X)
            ]
        )
    
    def _predict_cnn(self, X) -> np.ndarray:
        """Predicts labels for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predictions for.
        Returns
        -------
        y : array-like, shape = [n_instances]
            Predicted class labels.
        """
        rng = check_random_state(self.random_state)
        return np.array(
            [
                self.classes_[int(rng.choice(np.flatnonzero(prob == prob.max())))]
                for prob in self.predict_proba_cnn(X)
            ]
        )

    def _predict_proba(self, X) -> np.ndarray:
        """Predicts labels probabilities for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predict probabilities for.
        Returns
        -------
        y : array-like, shape = [n_instances, n_classes_]
            Predicted probabilities using the ordering in classes_.
        """
        dists = np.zeros((X.shape[0], self.n_classes_))

        # Call predict proba on each classifier, multiply the probabilities by the
        # classifiers weight then add them to the current HC1 probabilities
        dists = np.add(
            dists,
            self._stc.predict_proba(X) * (np.ones(self.n_classes_) * self.stc_weight_),
        )
        dists = np.add(
            dists,
            self._tsf.predict_proba(X) * (np.ones(self.n_classes_) * self.tsf_weight_),
        )
        dists = np.add(
            dists,
            self._rise.predict_proba(X)
            * (np.ones(self.n_classes_) * self.rise_weight_),
        )
        dists = np.add(
            dists,
            self._cboss.predict_proba(X)
            * (np.ones(self.n_classes_) * self.cboss_weight_),
        )

        # Make each instances probability array sum to 1 and return
        return dists / dists.sum(axis=1, keepdims=True)
    
    def _predict_proba_cnn(self, X) -> np.ndarray:
        """Predicts labels probabilities for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predict probabilities for.
        Returns
        -------
        y : array-like, shape = [n_instances, n_classes_]
            Predicted probabilities using the ordering in classes_.
        """
        dists = np.zeros((X.shape[0], self.n_classes_))

        # Call predict proba on each classifier, multiply the probabilities by the
        # classifiers weight then add them to the current HC1 probabilities
        dists = np.add(
            dists,
            self._stc.predict_proba(X) * (np.ones(self.n_classes_) * self.stc_weight_),
        )
        dists = np.add(
            dists,
            self._tsf.predict_proba(X) * (np.ones(self.n_classes_) * self.tsf_weight_),
        )
        dists = np.add(
            dists,
            self._rise.predict_proba(X)
            * (np.ones(self.n_classes_) * self.rise_weight_),
        )
        dists = np.add(
            dists,
            self._cboss.predict_proba(X)
            * (np.ones(self.n_classes_) * self.cboss_weight_),
        )
        dataset_MY = TSCDataset_MY(X)
        dataloader_MY = DataLoader(dataset_MY, batch_size=dataset_MY.__len__(), shuffle=False)
        for _, X_MY in enumerate(train_dataloader):
            cnn_pred = softmax(hc4.cnn(X_MY)).detach().numpy()
        dists = np.add(
            dists,
            cnn_pred
            * (np.ones(self.n_classes_) * self.cnn_weight_),
        )

        # Make each instances probability array sum to 1 and return
        return dists / dists.sum(axis=1, keepdims=True)

    @classmethod
    def get_test_params(cls, parameter_set="default"):
        """Return testing parameter settings for the estimator.
        Parameters
        ----------
        parameter_set : str, default="default"
            Name of the set of test parameters to return, for use in tests. If no
            special parameters are defined for a value, will return `"default"` set.
            For classifiers, a "default" set of parameters should be provided for
            general testing, and a "results_comparison" set for comparing against
            previously recorded results if the general set does not produce suitable
            probabilities to compare against.
        Returns
        -------
        params : dict or list of dict, default={}
            Parameters to create testing instances of the class.
            Each dict are parameters to construct an "interesting" test instance, i.e.,
            `MyClass(**params)` or `MyClass(**params[i])` creates a valid test instance.
            `create_test_instance` uses the first (or only) dictionary in `params`.
        """
        from sklearn.ensemble import RandomForestClassifier

        if parameter_set == "results_comparison":
            return {
                "stc_params": {
                    "estimator": RandomForestClassifier(n_estimators=3),
                    "n_shapelet_samples": 50,
                    "max_shapelets": 5,
                    "batch_size": 10,
                },
                "tsf_params": {"n_estimators": 3},
                "rise_params": {"n_estimators": 3},
                "cboss_params": {"n_parameter_samples": 5, "max_ensemble_size": 3},
            }
        else:
            return {
                "stc_params": {
                    "estimator": RandomForestClassifier(n_estimators=1),
                    "n_shapelet_samples": 5,
                    "max_shapelets": 5,
                    "batch_size": 5,
                },
                "tsf_params": {"n_estimators": 1},
                "rise_params": {"n_estimators": 1},
                "cboss_params": {"n_parameter_samples": 1, "max_ensemble_size": 1},
            }


In [33]:
hc = HIVECOTEV1_MY()

In [13]:
hc.fit(X_ECG, y_ECG)

HIVECOTEV1_MY()

In [14]:
hc.save("./HC_MY_BIG_ECG7_5classes")

<zipfile.ZipFile filename='HC_MY_BIG_ECG7_5classes.zip' mode='r'>

In [26]:
hc.load_from_path("/home/ironic/repos/TSC_NN_Experiments/HC_MY_BIG_ECG7_5classes.zip")

HIVECOTEV1_MY()

In [30]:
hc.stc_weight_ = 23
print(hc.stc_weight_)
hc.save("./test")

23


<zipfile.ZipFile filename='test.zip' mode='r'>

In [34]:
hc.load_from_path("./test.zip")

HIVECOTEV1_MY()

In [35]:
print(hc.stc_weight_)

0


In [23]:
predictions = hc.predict(X_ECG)

NotFittedError: This instance of HIVECOTEV1_MY has not been fitted yet; please call `fit` first.

In [17]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))

PRECISION:  1.0
RECALL:  1.0
F1:  1.0


In [18]:
df_test = pd.read_csv('mitbih_test.csv')
X_ECG_test = np.array(df_test.iloc[:, :-1])
print(len(X_ECG_test))
y_ECG_test = np.array(df_test.iloc[:,-1])
print(len(y_ECG_test))

21891
21891


In [19]:
predictions = hc.predict(X_ECG_test)

In [20]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))

PRECISION:  0.9440254360212906
RECALL:  0.6652379527504741
F1:  0.7544497475723144


In [6]:
hc1 = HIVECOTEV1()
hc1.fit(X5_small, np.asarray(list(map(lambda x: int(x), y5_small))))
predictions = hc1.predict(X5_test)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y5_test)), predictions))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y5_test)), predictions))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y5_test)), predictions))

PRECISION:  1.0
RECALL:  1.0
F1:  1.0


In [8]:
hc2 = HIVECOTEV1()
hc2.fit(X200_train, np.asarray(list(map(lambda x: int(x), y200_train))))
predictions = hc2.predict(X200_test)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y200_test)), predictions))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y200_test)), predictions))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y200_test)), predictions))

PRECISION:  0.8695652173913043
RECALL:  0.9375
F1:  0.9022556390977444


In [9]:
hc3 = HIVECOTEV1()
hc3.fit(X5000_train, np.asarray(list(map(lambda x: int(x), y5000_train))))
predictions = hc3.predict(X5000_test)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y5000_test)), predictions))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y5000_test)), predictions))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y5000_test)), predictions))

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [10]:
predictions

array([1, 1, 1, ..., 2, 2, 4])

In [17]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))

PRECISION:  0.9451111111111111
RECALL:  0.9451111111111111
F1:  0.9451111111111111


In [ ]:
hc2 = HIVECOTEV1()
hc2.fit(X200_train, y200_train)
predictions = hc2.predict(X200_test)
print("PRECISION: ", metrics.precision_score(y200_test, predictions))
print("RECALL: ",metrics.recall_score(y200_test, predictions))
print("F1: ", metrics.f1_score(y200_test, predictions))

In [ ]:
hc3 = HIVECOTEV1()
hc3.fit(X5000_train, y5000_train)
predictions = hc3.predict(X5000_test)
print("PRECISION: ", metrics.precision_score(y5000_test, predictions))
print("RECALL: ",metrics.recall_score(y5000_test, predictions))
print("F1: ", metrics.f1_score(y5000_test, predictions))

In [18]:
hc3.save("./HC_5000")

NameError: name 'hc3' is not defined

In [ ]:
list(map(lambda x: int(x), y5_small))

[1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1]

In [ ]:
def train_and_kfold_validate(X, y, num_splits = 5):
  kf = KFold(n_splits=num_splits)
  for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    # print(f"  Train: index={X.loc[train_index]}")
    hc1 = HIVECOTEV1()
    hc1.fit(X.loc[train_index], y[train_index])
    predictions = hc1.predict(X.loc[test_index])
    print("PRECISION: ", metrics.precision_score(y[test_index], predictions))
    print("RECALL: ",metrics.recall_score(y[test_index], predictions))
    print("F1: ", metrics.f1_score(y[test_index], predictions))

In [132]:
class TimeSeriesCnnModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.linear = nn.Sequential(
            nn.Linear(5888, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 5),
            #nn.Softmax()
            )
        
    def forward(self, xb):
        data = self.conv(xb)
        data = torch.flatten(data, start_dim=1)
        return self.linear(data)

In [133]:
tsc = TimeSeriesCnnModel()

In [84]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
def map_to_scores(x):
    if x == 0.0:
        return torch.tensor([1., 0., 0., 0., 0.])
    if x == 1.0:
        return torch.tensor([0., 1., 0., 0., 0.])
    if x == 2.0:
        return torch.tensor([0., 0., 1., 0., 0.])
    if x == 3.0:
        return torch.tensor([0., 0., 0., 1., 0.])
    if x == 4.0:
        return torch.tensor([0., 0., 0., 0., 1.])

class TSCDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return torch.tensor([self.x.iloc[i]]).float(), map_to_scores(float(self.y[i]))

In [78]:
dataset_test.__getitem__(0)

(tensor([[0.9084, 0.7839, 0.5311, 0.3626, 0.3663, 0.3443, 0.3333, 0.3077, 0.2967,
          0.3004, 0.3040, 0.3370, 0.3773, 0.3919, 0.4396, 0.4469, 0.4579, 0.4799,
          0.5128, 0.5348, 0.5861, 0.5861, 0.6044, 0.6337, 0.6154, 0.5824, 0.5641,
          0.5201, 0.4908, 0.4908, 0.4689, 0.4542, 0.4505, 0.4652, 0.4505, 0.4322,
          0.4249, 0.4103, 0.3919, 0.3700, 0.3736, 0.3663, 0.3187, 0.3114, 0.3260,
          0.3114, 0.3150, 0.3223, 0.3443, 0.3443, 0.3333, 0.3040, 0.3077, 0.2821,
          0.2711, 0.2857, 0.2637, 0.2747, 0.2821, 0.2674, 0.2784, 0.2894, 0.2930,
          0.2894, 0.2894, 0.2894, 0.2930, 0.2821, 0.2674, 0.2564, 0.2527, 0.2161,
          0.2234, 0.2381, 0.2161, 0.2674, 0.4176, 0.5824, 0.6154, 0.6227, 0.7619,
          0.8645, 1.0000, 0.9084, 0.6740, 0.3150, 0.1538, 0.1209, 0.0989, 0.0440,
          0.0220, 0.0220, 0.0293, 0.0000, 0.0110, 0.0293, 0.0147, 0.0220, 0.0440,
          0.0879, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000

In [17]:
df = pd.read_csv('mitbih_train.csv')
X_ECG = df.iloc[:, :-1]
print(len(X_ECG))
y_ECG = df.iloc[: ,-1]
print(len(y_ECG))

87553
87553


In [18]:
df = pd.read_csv('mitbih_test.csv')
X_ECG_test = df.iloc[:, :-1]
print(len(X_ECG_test))
y_ECG_test = df.iloc[: ,-1]
print(len(y_ECG_test))

21891
21891


In [85]:
dataset_train = TSCDataset(X_ECG, y_ECG)
dataset_test = TSCDataset(X_ECG_test, y_ECG_test)

In [93]:
batch_size = 128
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [134]:
from sklearn.metrics import f1_score, balanced_accuracy_score, recall_score
def get_f1_acc_recall(pred, target):
    #print(pred)
    #print(target)
    rec = recall_score(target, pred, average='macro')
    f1 = f1_score(target, pred, average='macro')
    #print(f"RECALL SCORE: {rec}")
    #print(f"ACCURACY SCORE: {balanced_accuracy_score(target, pred)}")
    #print(f"F1 SCORE: {f1}")
    return rec, f1

def evaluate(model, dataloader_test):
    lst = []
    targets = []
    model.eval()
    with torch.no_grad():
        for X, y in dataloader_test:
            pred = model(X)
            #print(pred)
            lst.append((torch.argmax(pred, dim=1)).numpy())
            #print(lst)
            targets.append(torch.argmax(y, dim=1).numpy())
    lst = np.concatenate(lst)
    #print(lst)
    #lst = [map_to_scores(x) for x in lst[0]]
    targets = np.concatenate(targets)
    #print(lst)
    #print(targets)
    return get_f1_acc_recall(lst, targets)

In [135]:
evaluate(tsc, test_dataloader)

(0.2, 0.0029383757311930354)

In [141]:
import tqdm
def train_loop(model, epochs, optim, loss_fn, dataloader_train, dataloader_test, n_batches):
    best_rec = 0
    best_f1 = 0
    for epoch in range(epochs):
        print("EPOCH: ", epoch)
        for i, (X, y) in tqdm.notebook.tqdm(enumerate(dataloader_train), total=n_batches):
            #print(X, y)
            pred = model(X)
            #print(pred)
            loss = loss_fn(pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
    
            if i % (n_batches//64) == 0:
                l= loss.item()
                print(f"train loss: {l:>5f}")
                
            if i != 0 and i % (n_batches//8) == 0: 
                rec, f1 = evaluate(model, dataloader_test)
                print("RECALL: ", rec)
                print("F1 SCORE: ", f1)
                if rec > best_rec: best_rec = rec
                if f1 > best_f1: best_f1 = f1
                if f1 < best_f1 and rec < best_rec:
                    print("EARLY STOPPING")
                    return
        model.train()

In [142]:
optim = torch.optim.Adam(tsc.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
train_loop(tsc, 10, optim, loss_fn, train_dataloader, test_dataloader, len(dataset_train)//batch_size)

EPOCH:  0


  0%|          | 0/684 [00:00<?, ?it/s]

train loss: 0.123301
train loss: 0.173309
train loss: 0.251348
train loss: 0.178053
train loss: 0.157526
train loss: 0.179611
train loss: 0.312158
train loss: 0.195368
train loss: 0.103936
RECALL:  0.7696066791986107
F1 SCORE:  0.8178229622847674
train loss: 0.029940
train loss: 0.172801
train loss: 0.072557
train loss: 0.226858
train loss: 0.249339
train loss: 0.115657
train loss: 0.102927
train loss: 0.060536
train loss: 0.131552
RECALL:  0.7676738654518621
F1 SCORE:  0.8202255973860971
train loss: 0.068407
train loss: 0.116860
train loss: 0.185388
train loss: 0.087154
train loss: 0.038840
train loss: 0.116457
train loss: 0.044985
train loss: 0.122246
RECALL:  0.7892853472315811
F1 SCORE:  0.8303347963887673
train loss: 0.216487
train loss: 0.105329
train loss: 0.150224
train loss: 0.138800
train loss: 0.075518
train loss: 0.077840
train loss: 0.242555
train loss: 0.056112
train loss: 0.164757
RECALL:  0.7894986680013885
F1 SCORE:  0.8312864301512093
train loss: 0.142661
train loss: 

In [ ]:
train_and_kfold_validate(X5_small, np.asarray(list(map(lambda x: int(x), y5_small))), num_splits = 3)

In [ ]:
train_and_kfold_validate(X200_train, y200_train)

In [ ]:
train_and_kfold_validate(X5000_train, y5000_train)

In [10]:
from datetime import datetime

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.utils import check_random_state

from sktime.classification.base import BaseClassifier
from sktime.classification.dictionary_based import ContractableBOSS
from sktime.classification.interval_based import (
    RandomIntervalSpectralEnsemble,
    TimeSeriesForestClassifier,
)
from sktime.classification.shapelet_based import ShapeletTransformClassifier

class HIVECOTEV1_MY(HIVECOTEV1):
    _tags = {
        "capability:multithreading": True,
        "classifier_type": "hybrid",
    }

    def __init__(
        self,
        stc_params=None,
        tsf_params=None,
        rise_params=None,
        cboss_params=None,
        verbose=0,
        n_jobs=1,
        random_state=None,
    ):
        self.stc_params = stc_params
        self.tsf_params = tsf_params
        self.rise_params = rise_params
        self.cboss_params = cboss_params

        self.verbose = verbose
        self.n_jobs = n_jobs
        self.random_state = random_state

        self.stc_weight_ = 0
        self.tsf_weight_ = 0
        self.rise_weight_ = 0
        self.cboss_weight_ = 0

        self.cnn_weight_ = 0

        self._stc_params = stc_params
        self._tsf_params = tsf_params
        self._rise_params = rise_params
        self._cboss_params = cboss_params
        self._stc = None
        self._tsf = None
        self._rise = None
        self._cboss = None

        self.cnn = None

        super(HIVECOTEV1, self).__init__()

    def _fit(self, X, y):
        """Fit HIVE-COTE 1.0 to training data.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The training data.
        y : array-like, shape = [n_instances]
            The class labels.
        Returns
        -------
        self :
            Reference to self.
        Notes
        -----
        Changes state by creating a fitted model that updates attributes
        ending in "_" and sets is_fitted flag to True.
        """
        # Default values from HC1 paper
        if self.stc_params is None:
            self._stc_params = {"transform_limit_in_minutes": 120}
        if self.tsf_params is None:
            self._tsf_params = {"n_estimators": 500}
        if self.rise_params is None:
            self._rise_params = {"n_estimators": 500}
        if self.cboss_params is None:
            self._cboss_params = {}

        # Cross-validation size for TSF and RISE
        cv_size = 10
        _, counts = np.unique(y, return_counts=True)
        min_class = np.min(counts)
        if min_class < cv_size:
            cv_size = min_class

        # Build STC
        self._stc = ShapeletTransformClassifier(
            **self._stc_params,
            save_transformed_data=True,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._stc.fit(X, y)

        if self.verbose > 0:
            print("STC ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find STC weight using train set estimate
        train_probs = self._stc._get_train_probs(X, y)
        train_preds = self._stc.classes_[np.argmax(train_probs, axis=1)]
        self.stc_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "STC train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("STC weight = " + str(self.stc_weight_))  # noqa

        # Build TSF
        self._tsf = TimeSeriesForestClassifier(
            **self._tsf_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._tsf.fit(X, y)

        if self.verbose > 0:
            print("TSF ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find TSF weight using train set estimate found through CV
        train_preds = cross_val_predict(
            TimeSeriesForestClassifier(
                **self._tsf_params, random_state=self.random_state
            ),
            X=X,
            y=y,
            cv=cv_size,
            n_jobs=self._threads_to_use,
        )
        self.tsf_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "TSF train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("TSF weight = " + str(self.tsf_weight_))  # noqa

        # Build RISE
        self._rise = RandomIntervalSpectralEnsemble(
            **self._rise_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._rise.fit(X, y)

        if self.verbose > 0:
            print("RISE ", datetime.now().strftime("%H:%M:%S %d/%m/%Y"))  # noqa

        # Find RISE weight using train set estimate found through CV
        train_preds = cross_val_predict(
            RandomIntervalSpectralEnsemble(
                **self._rise_params,
                random_state=self.random_state,
            ),
            X=X,
            y=y,
            cv=cv_size,
            n_jobs=self._threads_to_use,
        )
        self.rise_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "RISE train estimate ",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("RISE weight = " + str(self.rise_weight_))  # noqa

        # Build cBOSS
        self._cboss = ContractableBOSS(
            **self._cboss_params,
            random_state=self.random_state,
            n_jobs=self._threads_to_use,
        )
        self._cboss.fit(X, y)

        # Find cBOSS weight using train set estimate
        train_probs = self._cboss._get_train_probs(X, y)
        train_preds = self._cboss.classes_[np.argmax(train_probs, axis=1)]
        self.cboss_weight_ = accuracy_score(y, train_preds) ** 4

        if self.verbose > 0:
            print(  # noqa
                "cBOSS (estimate included)",
                datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
            )
            print("cBOSS weight = " + str(self.cboss_weight_))  # noqa

        return self

    def train_add_nn(self, model, epochs, optim, loss_fn, X, y, n_batches):
      
        for epoch in range(epochs):
            for i, (X, y) in tqdm.notebook.tqdm(enumerate(dataloader_train), total=n_batches):
            pred = model(X)
            loss = loss_fn(pred, map_to_scores(y))
            optim.zero_grad()
            loss.backward()
            optim.step()
    
            if i % (n_batches//3) == 0:
                l= loss.item()
                print(f"train loss: {l:>5f}")
                
        self.cnn = model

        # train_preds = self.cnn(X)
        # self.cnn_weight_ = accuracy_score(y, train_preds) ** 4

        # print("cBOSS weight = " + str(self.cnn_weight_))  # noqa


    def _predict(self, X) -> np.ndarray:
        """Predicts labels for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predictions for.
        Returns
        -------
        y : array-like, shape = [n_instances]
            Predicted class labels.
        """
        rng = check_random_state(self.random_state)
        return np.array(
            [
                self.classes_[int(rng.choice(np.flatnonzero(prob == prob.max())))]
                for prob in self.predict_proba(X)
            ]
        )

    def _predict_proba(self, X) -> np.ndarray:
        """Predicts labels probabilities for sequences in X.
        Parameters
        ----------
        X : 3D np.array of shape = [n_instances, n_dimensions, series_length]
            The data to make predict probabilities for.
        Returns
        -------
        y : array-like, shape = [n_instances, n_classes_]
            Predicted probabilities using the ordering in classes_.
        """
        dists = np.zeros((X.shape[0], self.n_classes_))

        # Call predict proba on each classifier, multiply the probabilities by the
        # classifiers weight then add them to the current HC1 probabilities
        dists = np.add(
            dists,
            self._stc.predict_proba(X) * (np.ones(self.n_classes_) * self.stc_weight_),
        )
        dists = np.add(
            dists,
            self._tsf.predict_proba(X) * (np.ones(self.n_classes_) * self.tsf_weight_),
        )
        dists = np.add(
            dists,
            self._rise.predict_proba(X)
            * (np.ones(self.n_classes_) * self.rise_weight_),
        )
        dists = np.add(
            dists,
            self._cboss.predict_proba(X)
            * (np.ones(self.n_classes_) * self.cboss_weight_),
        )
        dists = np.add(
            dists,
            self.cnn(X)
            * (np.ones(self.n_classes_) * self.cnn_weight_),
        )

        # Make each instances probability array sum to 1 and return
        return dists / dists.sum(axis=1, keepdims=True)

    @classmethod
    def get_test_params(cls, parameter_set="default"):
        """Return testing parameter settings for the estimator.
        Parameters
        ----------
        parameter_set : str, default="default"
            Name of the set of test parameters to return, for use in tests. If no
            special parameters are defined for a value, will return `"default"` set.
            For classifiers, a "default" set of parameters should be provided for
            general testing, and a "results_comparison" set for comparing against
            previously recorded results if the general set does not produce suitable
            probabilities to compare against.
        Returns
        -------
        params : dict or list of dict, default={}
            Parameters to create testing instances of the class.
            Each dict are parameters to construct an "interesting" test instance, i.e.,
            `MyClass(**params)` or `MyClass(**params[i])` creates a valid test instance.
            `create_test_instance` uses the first (or only) dictionary in `params`.
        """
        from sklearn.ensemble import RandomForestClassifier

        if parameter_set == "results_comparison":
            return {
                "stc_params": {
                    "estimator": RandomForestClassifier(n_estimators=3),
                    "n_shapelet_samples": 50,
                    "max_shapelets": 5,
                    "batch_size": 10,
                },
                "tsf_params": {"n_estimators": 3},
                "rise_params": {"n_estimators": 3},
                "cboss_params": {"n_parameter_samples": 5, "max_ensemble_size": 3},
            }
        else:
            return {
                "stc_params": {
                    "estimator": RandomForestClassifier(n_estimators=1),
                    "n_shapelet_samples": 5,
                    "max_shapelets": 5,
                    "batch_size": 5,
                },
                "tsf_params": {"n_estimators": 1},
                "rise_params": {"n_estimators": 1},
                "cboss_params": {"n_parameter_samples": 1, "max_ensemble_size": 1},
            }

In [11]:
hc4 = HIVECOTEV1_MY()
hc4.fit(X5_small, np.asarray(list(map(lambda x: int(x), y5_small))))

HIVECOTEV1_MY()